In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
# load IRIS dataset
dataset = pd.read_csv('iris.csv')

dataset[dataset["species"] == "Iris-setosa"] = 0
dataset[dataset["species"] == "Iris-versicolor"] = 1
dataset[dataset["species"] == "Iris-virginica"] = 2

train_X, test_X, train_y, test_y = train_test_split(np.asarray(dataset.iloc[:,:4].astype("float32")),
                                                    np.asarray(dataset["species"].astype("int64")),
                                                    test_size=0.2,
                                                    shuffle=True, 
                                                    random_state=31)

train_X = torch.tensor(train_X, dtype=torch.float).to(device)
test_X = torch.tensor(test_X, dtype=torch.float).to(device)
train_y = torch.tensor(train_y, dtype=torch.long).to(device)
test_y = torch.tensor(test_y, dtype=torch.long).to(device)

net = Net().to(device)

criterion = nn.CrossEntropyLoss()# cross entropy loss

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

for epoch in range(1000):
    out = net(train_X)
    loss = criterion(out, train_y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print (f"number of epoch: {epoch} loss: {loss.item()}")

predict_out = net(test_X)
_, predict_y = torch.max(predict_out, 1)

if (device == 'cuda'):
    print(f"prediction accuracy: {accuracy_score(test_y.cpu().data, predict_y.cpu().data)}")

    print(f"macro precision: {precision_score(test_y.cpu().data, predict_y.cpu().data, average='macro')}")
    print(f"micro precision: {precision_score(test_y.cpu().data, predict_y.cpu().data, average='micro')}")
    print(f"macro recall: {recall_score(test_y.cpu().data, predict_y.cpu().data, average='macro')}")
    print (f"micro recall: {recall_score(test_y.cpu().data, predict_y.cpu().data, average='micro')}")   

else:
    print(f"prediction accuracy: {accuracy_score(test_y.data, predict_y.data)}")

    print(f"macro precision: {precision_score(test_y.data, predict_y.data, average='macro')}")
    print(f"micro precision: {precision_score(test_y.data, predict_y.data, average='micro')}")
    print(f"macro recall: {recall_score(test_y.data, predict_y.data, average='macro')}")
    print (f"micro recall: {recall_score(test_y.data, predict_y.data, average='micro')}")